In [1]:
#importing the required scripts
from Scripts.ProcessingEmbeddings import *
import Scripts.utils as utils
from Scripts.HardDebias import *

#Importing the required packages
import numpy as np
import pandas as pd



In [2]:
#Creating an embeddings object: 400k words, 50 dimensions
glove=Embeddings('Data/glove-wiki-gigaword-300.txt', gensim=False)




Loading Data/glove-wiki-gigaword-300.txt embeddings
vectors shape: (400000, 300), word2idx length: 400000, vocab length: 400000


In [3]:
# From the embeddings object, get the vectors, the word2idx dictionary, the vocab list, and the dict_vectors dictionary
# Because the gensim embeddings carry no information on the file, we need to use the built-in function from gensim to get the vocab in descending frequency.
glove.model.sort_by_descending_frequency()
vectors = glove.vectors
word2idx = glove.word2idx
vocab = glove.words
dict_vectors = glove.get_word_vector_dict()

#print the first 20 words in the vocab
print(vocab[:20])

#Print the shape of the vectors
print("vectors shape", vectors.shape)

#Print a boolean to check if there are any NaNs in the vectors
print("Missing values in vectors?", np.isnan(vectors).any())




['the', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', "'s", 'for', '-', 'that', 'on', 'is', 'was', 'said', 'with', 'he', 'as']
vectors shape (400000, 300)
Missing values in vectors? False


In [4]:
#Removing puntuation and numbers from the embeddings
vocab_cleaned, vectors_cleaned, word2idx_cleaned, dict_vec_cleaned = glove.limit_vocab(vectors, word2idx, vocab)



100%|██████████| 400000/400000 [00:00<00:00, 618093.77it/s]


Size of limited vocabulary: 327185


  ## Hard-Debias Algorithm

  #### Preliminaries

In [5]:
#Getting the definitional sets to calculate afterwards the gender direction. The first 10 gender sets were proposed by Bolukbasi et al. (2016)
#Definitional sets for race where proposed by Manzini et al. in Multiclass debiasing of embeddings: https://github.com/TManzini/DebiasMulticlassWordEmbedding/blob/master/Debiasing/data/vocab/race_attributes_optm.json

def_sets = {
    "gender": [
        ['she', 'he'], ['herself', 'himself'], 
        ['her', 'his'], ['daughter', 'son'], ['girl', 'boy'],
        ['mother', 'father'], ['woman', 'man'], ['mary', 'john'],
        ['gal', 'guy'], ['female', 'male'], ['aunt', 'uncle']],

    "race": [
        ["black", "caucasian", "asian", "hispanic"],
      		["african", "caucasian", "asian", "hispanic"],
      		["black", "white", "asian", "latino"],
      		["africa", "europe", "asia", "mexico"],
      		["africa", "america", "china", "latin-america"],
    ]
}

#Equalizing pairs for gender debiasing were first published by Bolukbasi et al. in https://github.com/tolga-b/debiaswe/blob/master/data/equalize_pairs.json
# Equalizing sets for race where defined by Manzini as equal to the defining set (Manzini et al., 2019.p.3)
equalizing_lists = {
    "gender": [
        ["monastery", "convent"], ["spokesman", "spokeswoman"], 
        ["Catholic_priest", "nun"], ["Dad", "Mom"], ["Men", "Women"],
        ["councilman", "councilwoman"], ["grandpa", "grandma"], 
        ["grandsons", "granddaughters"], ["prostate_cancer", "ovarian_cancer"],
        ["testosterone", "estrogen"], ["uncle", "aunt"], 
        ["wives", "husbands"], ["Father", "Mother"], ["Grandpa", "Grandma"],
        ["He", "She"], ["boy", "girl"], ["boys", "girls"], ["brother", "sister"], 
        ["brothers", "sisters"], ["businessman", "businesswoman"],
        ["chairman", "chairwoman"], ["colt", "filly"], ["congressman","congresswoman"], 
        ["dad", "mom"], ["dads", "moms"], ["dudes", "gals"],
        ["ex_girlfriend", "ex_boyfriend"], ["father", "mother"],
        ["fatherhood", "motherhood"], ["fathers", "mothers"], ["fella", "granny"],
        ["fraternity", "sorority"], ["gelding", "mare"], ["gentleman", "lady"], 
        ["gentlemen", "ladies"], ["grandfather", "grandmother"],
        ["grandson", "granddaughter"], ["he", "she"], ["himself", "herself"], 
        ["his", "her"], ["king", "queen"], ["kings", "queens"],
        ["male", "female"], ["males", "females"], ["man", "woman"],
        ["men", "women"], ["nephew", "niece"], ["prince", "princess"],
        ["schoolboy", "schoolgirl"], ["son", "daughter"], ["sons", "daughters"], 
        ["twin_brother", "twin_sister"]]}

#Words taken from Wang et al. to enrich the equalizing pairs
female_vocab = ['countrywoman',  'witches',  'maidservant',  'mothers',  'diva',  'actress',  'spinster',  'mama',  'duchesses',  'countrywomen',  'hostesses',  'suitors',  'menopause',  'clitoris',  'princess',  'governesses',  'abbess',  'women',  'widow',  'ladies',  'sorceresses',  'madam',  'brides',  'baroness',  'niece',  'widows',  'lady',  'sister',  'brides',  'nun',  'obstetrics',  'her',  'marchioness',  'princesses',  'empresses',  'mare',  'chairwoman',  'convent',  'priestesses',  'girlhood',  'ladies',  'queen',  'gals',  'mommies',  'maid',  'spokeswoman',  'seamstress',  'cowgirls',  'chick',  'spinsters',  'empress',  'mommy',  'gals',  'enchantress',  'gal',  'motherhood',  'estrogen',  'godmother',  'strongwoman',  'goddess',  'matriarch',  'aunt',  'chairwomen',  'maam',
                'sisterhood',  'hostess',  'estradiol',  'wife',  'mom',  'stewardess',  'females',  'spokeswomen',  'ma',  'belle',  'minx',  'maiden',  'witch',  'miss',  'nieces',  'mothered',  'cow',  'belles',  'granddaughter',  'fiancees',  'stepmothers',  'grandmothers',  'schoolgirl',  'hen',  'granddaughters',  'bachelorette',  'camerawoman',  'moms',  'her',  'mistress',  'lass',  'policewoman',  'nun',  'actresses',  'saleswomen',  'girlfriend',  'councilwoman',  'lady',  'stateswoman',  'maternal',  'lass',  'landlady',  'ladies',  'wenches',  'sorority',  'duchess',  'ballerina',  'chicks',  'fiancee',  'fillies',  'wives',  'she',  'businesswoman',  'masseuses',  'heroine',  'doe',  'girlfriends',  'queens',  'sisters',  'stepmother',  'daughter',  'cowgirl',  'daughters',  'soprano',
                'saleswoman',  'mistress',  'nuns',  'headmistresses',  'lasses',  'congresswoman',  'housewife',  'priestess',  'abbesses',  'toque',  'sororities',  'stewardesses',  'filly',  'czarina',  'stepdaughters',  'herself',  'girls',  'lionesses',  'lady',  'vagina',  'hers',  'masseuse',  'cows',  'aunts',  'wench',  'toques',  'wife',  'lioness',  'sorceress',  'mother',  'lesbians',  'female',  'waitresses',  'ovum',  'ovary',  'stepdaughter',  'businesswomen',  'heiress',  'waitress',  'headmistress',  'woman',  'governess',  'bride',  'grandma',  'bride',  'gal',  'lesbian',  'ladies',  'girl',  'grandmother',  'mare',  'hens',  'nuns',  'maidservants',  'heroines']
male_vocab = ['countryman',  'wizards',  'manservant',  'fathers',  'divo',  'actor',  'bachelor',  'papa',  'dukes',  'countrymen',  'hosts',  'airmen',  'andropause',  'penis',  'prince',  'governors',  'abbot',  'men',  'widower',  'gentlemen',  'sorcerers',  'sir',  'bridegrooms',  'baron',  'nephew',  'widowers',  'lord',  'brother',  'grooms',  'priest',  'andrology',  'his',  'marquis',  'princes',  'emperors',  'stallion',  'chairman',  'monastery',  'priests',  'boyhood',  'fellas',  'king',  'dudes',  'daddies',  'manservant',  'spokesman',  'tailor',  'cowboys',  'dude',  'bachelors',  'emperor',  'daddy',  'guys',  'enchanter',  'guy',  'fatherhood', 
                'androgen',  'godfather',  'strongman',  'god',  'patriarch',  'uncle',  'chairmen',  'sir',  'brotherhood',  'host',  'testosterone',  'husband',  'dad',  'steward',  'males',  'spokesmen',  'pa',  'beau',  'stud',  'bachelor',  'wizard',  'sir',  'nephews',  'fathered',  'bull',  'beaus',  'grandson',  'fiances',  'stepfathers',  'grandfathers',  'schoolboy',  'rooster',  'grandsons',  'bachelor',  'cameraman',  'dads',  'him',  'master',  'lad',  'policeman',  'monk',  'actors',  'salesmen',  'boyfriend',  'councilman',  'fella',  'statesman',  'paternal',  'chap',  'landlord',  'lords',  'blokes',  'fraternity',  'duke',  'dancer',  'dudes',  'fiance',
                'colts',  'husbands',  'he',  'businessman',  'masseurs',  'hero',  'deer',  'boyfriends',  'kings',  'brothers',  'stepfather',  'son',  'cowboy',  'sons',  'baritone',  'salesman',  'paramour',  'monks',  'headmasters',  'lads',  'congressman',  'househusband',  'priest',  'abbots',  'beard',  'fraternities',  'stewards',  'colt',  'czar',  'stepsons',  'himself',  'boys',  'lions',  'gentleman',  'penis',  'his',  'masseur',  'bulls',  'uncles',  'bloke',  'beards',  'hubby',  'lion',  'sorcerer',  'father',  'gays',  'male',  'waiters',  'sperm',  'prostate',  'stepson',  'businessmen',  'heir',  'waiter',  'headmaster',  'man',  'governor',  'bridegroom', 
                'grandpa',  'groom',  'dude',  'gay',  'gents',  'boy',  'grandfather',  'gelding',  'roosters',  'priests',  'busboy',  'heros']

#added the gendered pairs to the equalizing list (equalizing_lists['gender'] if the pairs are not there already
gendered_pairs = list(zip(male_vocab, female_vocab))
list_pairs = [tuple for tuple in gendered_pairs if tuple not in equalizing_lists['gender']]
equalizing_lists['gender'] = equalizing_lists['gender']+list_pairs

#Some of the words were taken from the analogies' templates from Cheng and Manzini.
#The list is not the same, however, because some of the words were not neutral, but carried some
#relation to the social categories.
neutral_words = ["manager", "executive", "doctor", "lawyer", "programmer",
                 "scientist", "soldier", "supervisor", "rancher", "janitor",
                 "firefighter", "officer", "secretary", "nurse", "clerk", "artist",
                 "homemaker", "dancer", "singer", "librarian", "maid", "hairdresser", "stylist",
                 "receptionist", "counselor", "leader", "farmer",
                 "engineer", "laborer", "teacher",
                 "slave", "musician", "runner", "criminal", "homeless",
                 "greedy", "cheap", "hairy", "liberal",
                 "judgemental", "conservative", "familial",
                 "violent", "terrorist", "dirty", "uneducated", "educated"]


#However, also the vocabulary without the gendered words from the list can be conceived as neutral, according to Bolukbasi et al.




In [6]:
#Preparing the definite sets for debiasing
def_set_gender=utils.prepare_def_sets_subspace(def_sets["gender"])




  ## What's the better way to find the Bias direction
  The success of hard debiasing algorithms lies on their capacity to find the appropriate bias direction. To have an idea of how good the algorithms are on that task, we'll count how many of the explicitly gendered words in the lists are identified by the gender direction.

In [7]:
#Find the words in the female_vocab that are also in the embeddings
female_words_emb = [
    word for word in female_vocab if word in dict_vec_cleaned.keys()]
male_words_emb = [
    word for word in male_vocab if word in dict_vec_cleaned.keys()]

print('Number of female words in embeddings:', len(female_words_emb))
print('Number of male words in embeddings:', len(male_words_emb))





Number of female words in embeddings: 182
Number of male words in embeddings: 183


  Getting the gender direction to be able to identify words that are biase. The first one centralizes the definite sets, while the second one doesn't.

In [8]:
#getting the gender directions
gen_dir_centralized = identify_bias_subspace(
    dict_vec_cleaned, def_set_gender, 1, centralizing=True)
gen_dir = identify_bias_subspace(
    dict_vec_cleaned, def_set_gender, 1, centralizing=False)

#flattening them
gen_dir_centralized_flat = np.squeeze(gen_dir_centralized)
gen_dir_flat = np.squeeze(gen_dir)




Length of vectors set: 22
Running PCA with 1 components
Length of vectors set: 11
Running PCA with 1 components


In [9]:
#Compute the similarity to the bias direction
from Scripts.Evaluation import compute_gender_simple_bias, compute_similarity_to_bias_direction
similarity=compute_similarity_to_bias_direction(dict_vec_cleaned, gen_dir_flat)
similarity_centralized=compute_similarity_to_bias_direction(dict_vec_cleaned, gen_dir_centralized_flat)
simple_gender_bias=compute_gender_simple_bias(dict_vec_cleaned, dict_vec_cleaned['he'], dict_vec_cleaned['she'])



In [10]:
#count how many female_words_emb and male_words_emb are in the biased_words
def count_gendered_words_in_most_biased(female_words, male_words, biased_words):
    count_female = 0
    count_male = 0
    data = {}
    for word in set(biased_words):
        if word in set(female_words):
            count_female += 1
        if word in set(male_words):
            count_male += 1

    data.update({'number_words': len(biased_words), 'count_female': count_female,
                 'recall_female': count_female/len(female_words),
                 'precision_female': count_female/len(biased_words),
                 'count_male': count_male, 'recall_male': count_male/len(male_words),
                 'precision_male': count_male/len(biased_words)})
    return data

#function to get a dictionary with the counts for experiments with different number of words
def get_counts_gendered_words_per_direction(similarity, female_words, male_words,  n_words):

    dict_results = {}
    last_biased_words = []
    for i in range(1, 11):
        number_words = int(n_words*i/100)
        #getting the most biased words and words that are neutral
        biased_words, _ = utils.get_most_biased_words_similarity(
            similarity, number_words)
        dict_counts = count_gendered_words_in_most_biased(
            female_words, male_words, biased_words)
        #add the dict_counts to the dict_results
        dict_results.update({str(i)+'%': dict_counts})
        last_biased_words = biased_words
    return dict_results, last_biased_words




In [11]:
#Getting the counts for the different experiments
dict_similarity, biased_words_similarity = get_counts_gendered_words_per_direction(
    similarity, female_words_emb,male_words_emb, n_words=len(vocab_cleaned))
dict_similarity_cent, biased_words_similarity_cent = get_counts_gendered_words_per_direction(
    similarity_centralized, female_words_emb, male_words_emb, n_words=len(vocab_cleaned))
dict_similarity_simple, biased_words_simple = get_counts_gendered_words_per_direction(
    simple_gender_bias, female_words_emb, male_words_emb, n_words=len(vocab_cleaned))




In [12]:
#getting all the dictionaries into one dataframe
df_similarity = pd.DataFrame.from_dict(dict_similarity, orient='index')
df_similarity['similarity']='not_centralized'
df_similarity_cent = pd.DataFrame.from_dict(dict_similarity_cent, orient='index')
df_similarity_cent['similarity']='centralized'
df_similarity_simple = pd.DataFrame.from_dict(dict_similarity_simple, orient='index')
df_similarity_simple['similarity']='simple'

#merging the dataframes
df_similarity_all = pd.concat([df_similarity, df_similarity_cent, df_similarity_simple], axis=0)

#saving the dataframe
#df_similarity_all.to_csv('Data/df_similarity_all.csv')



  It looks that the centralized algorithm gets more balanced results than the non-centralized one with respect to the interpretable baseline. Interestingly, all directions are very bad at picking male specific words.

  #### Let's plot the results:

  First let's get the most biased words by the simple metric and compares the scores: to see which gender direction allow to get a better sense of which words are feminine and which are masculine:

In [13]:
#getting the specific df's of bias scores
df_female=utils.get_df_bias_all_scores(female_words_emb, similarity, similarity_centralized,simple_gender_bias)
df_male=utils.get_df_bias_all_scores(male_words_emb, similarity, similarity_centralized,simple_gender_bias)

#sort the dataframe according to the simple_gender_bias column
most_female_bias=df_female.sort_values(by=['simple_bias_score'], ascending=False, inplace=True)
most_male_bias=df_male.sort_values(by=['simple_bias_score'], ascending=True, inplace=True)

#plotting the most biased words
from Scripts.Visualization import plot_top_biased_words
plot_top_biased_words(df_female, 'Most biased female words',n_words=20 )
plot_top_biased_words(df_male, 'Most biased male words', n_words=20, )




  Let's print the most biased words (ie. the words that have a largest component along the gender direction) in all cases:

In [14]:

#getting the most biased words and words that are neutral in the non-centralized case
biased_words, neutral_words=utils.get_most_biased_words_similarity(similarity, n_words=40000)
count_gendered_words_in_most_biased(
    female_words_emb,male_words_emb,biased_words)

#getting the most biased words and words that are neutral in the centralized case
biased_words_centralized, neutral_words_centralized = utils.get_most_biased_words_similarity(
    similarity_centralized, n_words=40000)
count_gendered_words_in_most_biased(
    female_words_emb, male_words_emb, biased_words_centralized)

#getting the most biased words and words that are neutral in the baseline
_, _, female_simple, male_simple, _ = utils.getting_biased_words(
    simple_gender_bias, def_sets['gender'], 20000, word2idx)

count_gendered_words_in_most_biased(female_words_emb, male_words_emb,female_simple+male_simple )



{'number_words': 40000,
 'count_female': 117,
 'recall_female': 0.6428571428571429,
 'precision_female': 0.002925,
 'count_male': 80,
 'recall_male': 0.4371584699453552,
 'precision_male': 0.002}

In [15]:
#turn the similarity scores into a dataframe to find the 20 words with the highest similarity score
import pandas as pd
df_similarity = pd.DataFrame.from_dict(similarity, orient='index')
df_similarity = df_similarity.rename(columns={0: 'similarity_score'})
df_similarity.sort_values(by=['similarity_score'],
                          ascending=False, inplace=True)
df_similarity_female = df_similarity.head(20)
df_similarity_male = df_similarity.tail(20)

print('Top biased words according to the similarity score:', biased_words[:20])

#turn the centralized similarity scores into a dataframe to find the 20 words with the highest similarity score
df_similarity_centralized = pd.DataFrame.from_dict(
    similarity_centralized, orient='index')
df_similarity_centralized = df_similarity_centralized.rename(
    columns={0: 'similarity_score'})
df_similarity_centralized.sort_values(
    by=['similarity_score'], ascending=False, inplace=True)
df_centralized_female = df_similarity_centralized.head(20)
df_centralized_male = df_similarity_centralized.tail(20)

print('Top biased words according to the centralized similarity score:',
      biased_words_centralized[:20])

#simple_biased = {word: abs(sim) for word, sim in simple_gender_bias.items()}
df_simple_bias = pd.DataFrame.from_dict(simple_gender_bias, orient='index')
df_simple_bias = df_simple_bias.rename(columns={0: 'simple_bias_score'})
df_simple_bias.sort_values(
    by=['simple_bias_score'], ascending=False, inplace=True)
df_simple_fem = df_simple_bias.head(20)
df_simple_masc = df_simple_bias.tail(20)
print('Top biased words simple_score', list(
    df_simple_bias.head(10).index)+list(df_simple_bias.tail(10).index))




Top biased words according to the similarity score: ['gal', 'guy', 'thing', 'pretty', 'everyone', 'really', 'woman', 'somebody', 'everybody', 'she', 'tell', 'girl', 'else', 'you', 'something', 'guys', 'someone', 'einsatzgruppen', 'always', 'maybe']
Top biased words according to the centralized similarity score: ['schorpen', 'lactating', 'kuppinger', 'actress', 'herself', 'pregnant', 'millerwise', 'jeria', 'selecky', 'okitundu', 'businesswoman', 'chromatid', 'mary', 'sasami', 'john', 'hamoaze', 'congresswoman', 'tallchief', 'heroine', 'diva']
Top biased words simple_score ['her', 'okitundu', 'countrywoman', 'actress', 'diva', 'schorpen', 'meye', 'herself', 'svetlana', 'jeria', 'mlb', 'football', 'defensive', 'parcells', 'chiefs', 'nfl', 'cardinals', 'himself', 'his', 'he']
